Discover Mechanics
==========

Let's do some Monte Carlos for Discover as well.  Some ones to try:

* Tomb Spider (Discover a Beast)
* Jewled Scarab (Discover a 3 Cost Card)
* Dark Peddler (Discover a 1-Cost Card - Warlock Only)
* Ethereal Conjurer (Discover a Spell - Mage Only)

In [20]:
# Load up all the cards
from helpers.data import load_dataframe, player_classes
from helpers.discover import discover
from collections import Counter
import numpy as np

df = load_dataframe()

In [7]:
# Tomb Spider (Discover a Beast)
# First, let's filter down to just the beasts
beasts = df[df.race == "Beast"]

# let's get the initial optimized dataframes for each class
beasts_by_class = dict()
for cls in player_classes:
    discarded_best, discarded_picks, optimized_df = discover(cls, beasts)
    beasts_by_class[cls] = optimized_df
    
# our function for monte-carloing this
def tomb_spider_monte_carlo(cls, n=10000):
    best_counter = Counter()
    picks_counter = Counter()
    for i in range(n):
        best, picks, discard = discover(cls, beasts_by_class[cls], needs_weights=False)
        best_counter.update([best["name"]])
        picks_counter.update(picks["name"].values)
    return best_counter, picks_counter

In [15]:
best, picks = tomb_spider_monte_carlo("Hunter", n=10000)
best.most_common(30)

[(u'Savannah Highmane', 1390),
 (u'King Krush', 1265),
 (u"Gahz'rilla", 1106),
 (u"King's Elekk", 926),
 (u'Dreadscale', 829),
 (u'Webspinner', 658),
 (u'Stampeding Kodo', 332),
 (u'Core Rager', 318),
 (u'Scavenging Hyena', 292),
 (u'Haunted Creeper', 265),
 (u'Stranglethorn Tiger', 226),
 (u'Maexxna', 223),
 (u'King of Beasts', 191),
 (u'Emperor Cobra', 181),
 (u'Tomb Spider', 165),
 (u'King Mukla', 148),
 (u'Jeweled Scarab', 147),
 (u'Tundra Rhino', 143),
 (u'Dire Wolf Alpha', 129),
 (u'Captured Jormungar', 117),
 (u'Bloodfen Raptor', 114),
 (u'Lost Tallstrider', 111),
 (u'River Crocolisk', 108),
 (u'Jungle Panther', 107),
 (u'Armored Warhorse', 100),
 (u'The Beast', 98),
 (u'Timber Wolf', 58),
 (u'Ironfur Grizzly', 49),
 (u'Ironbeak Owl', 48),
 (u'Stonetusk Boar', 34)]

In [32]:
# The Average Best Score is Actually much better than the Average Score
scores = []
for card, count in best.items():
    score = beasts[beasts["name"] == card].iloc[0].Hunter_Tier
    scores.extend([score] * count)

avg_best_score = np.mean(scores)
avg_best_score, beasts_by_class["Hunter"].Hunter_Tier.mean()

(74.2864, 54.358974358974358)